In [169]:
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import cv2
import numpy as np
import os
import seaborn as sns
import PIL
from PIL import Image,ImageFile


In [170]:
import tensorflow as tf
import keras
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.layers import BatchNormalization,Input
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,InputLayer,Input,UpSampling2D
from keras_preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image,sequence
from skimage.io import imsave
from keras.callbacks import TensorBoard
from skimage.color import rgb2lab,lab2rgb,gray2rgb
from skimage.transform import resize
import tensorflow as tf
import keras
from cv2 import resize

In [171]:
path = 'images/colorization/'


In [172]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

#Resize images, if needed
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=340, 
                                          class_mode=None)
X_train, X_test = train_test_split(train[0], test_size=0.2,random_state=1)
print(X_test.shape)

Found 16 images belonging to 1 classes.
(4, 256, 256, 3)


In [173]:
X =[]
Y =[]
for img in X_train:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(12, 256, 256, 1)
(12, 256, 256, 2)


In [174]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_127 (Conv2D)         (None, 128, 128, 64)      640       
                                                                 
 conv2d_128 (Conv2D)         (None, 128, 128, 128)     73856     
                                                                 
 conv2d_129 (Conv2D)         (None, 64, 64, 128)       147584    
                                                                 
 conv2d_130 (Conv2D)         (None, 64, 64, 256)       295168    
                                                                 
 conv2d_131 (Conv2D)         (None, 32, 32, 256)       590080    
                                                                 
 conv2d_132 (Conv2D)         (None, 32, 32, 512)       1180160   
                                                                 
 conv2d_133 (Conv2D)         (None, 32, 32, 512)      

In [175]:
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()
model.save("nikka")


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_127 (Conv2D)         (None, 128, 128, 64)      640       
                                                                 
 conv2d_128 (Conv2D)         (None, 128, 128, 128)     73856     
                                                                 
 conv2d_129 (Conv2D)         (None, 64, 64, 128)       147584    
                                                                 
 conv2d_130 (Conv2D)         (None, 64, 64, 256)       295168    
                                                                 
 conv2d_131 (Conv2D)         (None, 32, 32, 256)       590080    
                                                                 
 conv2d_132 (Conv2D)         (None, 32, 32, 512)       1180160   
                                                                 
 conv2d_133 (Conv2D)         (None, 32, 32, 512)      

INFO:tensorflow:Assets written to: nikka\assets


INFO:tensorflow:Assets written to: nikka\assets


In [176]:
model.fit(X,Y,validation_split=0.1, epochs=5, batch_size=16)

Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.1144 - accuracy: 0.4045 - val_loss: 0.8041 - val_accuracy: 0.8393
Epoch 2/5
1/1 [==============================] - 3s 3s/step - loss: 0.8451 - accuracy: 0.7964 - val_loss: 0.8850 - val_accuracy: 0.8448
Epoch 3/5
1/1 [==============================] - 3s 3s/step - loss: 0.9520 - accuracy: 0.7433 - val_loss: 0.8721 - val_accuracy: 0.7382
Epoch 4/5
1/1 [==============================] - 3s 3s/step - loss: 0.9270 - accuracy: 0.5776 - val_loss: 1.1856 - val_accuracy: 0.1564
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 1.1011 - accuracy: 0.2032 - val_loss: 1.0843 - val_accuracy: 0.1844


In [177]:
X_tests =[]
Y_tests =[]
for img in X_test:
  try:
      lab = rgb2lab(img)
      X_tests.append(lab[:,:,0]) 
      Y_tests.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X_tests = np.array(X_tests)
Y_tests= np.array(Y_tests)
X_tests = X_tests.reshape(X_tests.shape+(1,)) #dimensions to be the same for X and Y
print(X_tests.shape)
print(Y_tests.shape)

(4, 256, 256, 1)
(4, 256, 256, 2)


In [180]:
i=0
for img in X_test[i]:
    lab = rgb2lab(X_test[i])
    #print(lab.shape)
    #print(lab.shape)
    l= lab[:,:,0]
    print(l.shape)
    lo= l.reshape(l.shape+(1,))
    print(lo.shape)
    ab = model.predict(lo)
    #print(vggpred.shape)
    ab = ab*128
    print(ab.shape)
    result= np.zeros((256, 256, 3))
    result[:,:,0] = l
    result[:,:,1:] = ab 
    plt.imshow(lab2rgb(result))
    imsave('C:/Users/Acer/OneDrive/Desktop/New folder (2)/New folder/jpt/model1 %i.jpg'%i, lab2rgb(result))
    i=i+1

(256, 256)
(256, 256, 1)
8/8 [==============================] - 1s 93ms/step
(256, 256, 8, 2)


ValueError: could not broadcast input array from shape (256,256,8,2) into shape (256,256,2)